# Clean the Property Data Gathered

## Import Packages

In [18]:
# Web - Scraping and API Requests
import requests
from httpx import AsyncClient, Response
from parsel import Selector
import parsel
import jmespath
import asyncio

# Data Manipulation and Analysis
import pandas as pd
from pprint import pprint 
import json
from typing import List
from typing import TypedDict

# Database Connection
from sqlalchemy import create_engine
from sqlalchemy import inspect, text

# File and System Operations
import os
import sys

In [19]:
pd.set_option('display.max_columns', None) # Display all columns in any given DataFrame

In [20]:
# This allows one to reload the custom package without having to install it again
%load_ext autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
# this allows one to reload the custom package without having to install it again
%autoreload 1

sys.path.insert(0,'../src/')

# Import the custom package and sub-packages
%aimport rental_utils
%aimport rental_utils.functions
%aimport rental_utils.sql_queries

In [22]:
# Load SQL Magic for Jupyter Notebooks
%load_ext sql
%config SqlMagic.displaylimit = None
%config SqlMagic.autocommit=True # for engines that do not support autommit

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


displaylimit: Value None will be treated as 0 (no limit)

## Clean the data for selected urls

In [23]:
with open("../data/rightmove_properties.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [24]:
display(data[0])

{'id': 163153346,
 'bedrooms': 4,
 'bathrooms': 4,
 'numberOfImages': 15,
 'numberOfFloorplans': 1,
 'numberOfVirtualTours': 0,
 'summary': 'A spacious four bedroom lateral apartment to rent located near Holland Park. Luxurious and spacious, this exquisite four bedroom lateral apartment offers an unparalleled living experience in this prestigious Kensington block. Boasting elegance and comfort, this residence features: Air conditioni...',
 'displayAddress': 'Warwick Gardens, London, W14.',
 'countryCode': 'GB',
 'location': {'latitude': 51.495757, 'longitude': -0.202902},
 'propertyImages': {'images': [{'url': '66k/65890/163153346/65890_KEQ012539097_IMG_00_0000.jpeg',
    'caption': None,
    'srcUrl': 'https://media.rightmove.co.uk:443/dir/crop/10:9-16:9/66k/65890/163153346/65890_KEQ012539097_IMG_00_0000_max_476x317.jpeg'},
   {'url': '66k/65890/163153346/65890_KEQ012539097_IMG_01_0000.jpeg',
    'caption': None,
    'srcUrl': 'https://media.rightmove.co.uk:443/dir/crop/10:9-16:9/66k/

### Normalise th JSON so that it turns into a dataframe

In [25]:
data_norm = pd.json_normalize(data, max_level = 1)
data_norm.head(1)

,id,bedrooms,bathrooms,numberOfImages,numberOfFloorplans,numberOfVirtualTours,summary,displayAddress,countryCode,propertySubType,premiumListing,featuredProperty,distance,transactionType,commercial,development,residential,students,auction,feesApply,feesApplyText,displaySize,showOnMap,propertyUrl,contactUrl,staticMapUrl,channel,firstVisibleDate,keywords,keywordMatchType,saved,hidden,onlineViewingsAvailable,hasBrandPlus,displayStatus,enquiredTimestamp,enquiryAddedTimestamp,enquiryCalledTimestamp,heading,isRecent,enhancedListing,addedOrReduced,formattedBranchName,formattedDistance,propertyTypeFullDescription,location.latitude,location.longitude,propertyImages.images,propertyImages.mainImageSrc,propertyImages.mainMapImageSrc,listingUpdate.listingUpdateReason,listingUpdate.listingUpdateDate,price.amount,price.frequency,price.currencyCode,price.displayPrices,customer.branchId,customer.brandPlusLogoURI,customer.contactTelephone,customer.branchDisplayName,customer.branchName,customer.brandTradingName,customer.branchLandingPageUrl,customer.development,customer.showReducedProperties,customer.commercial,customer.showOnMap,customer.enhancedListing,customer.developmentContent,customer.buildToRent,customer.buildToRentBenefits,customer.brandPlusLogoUrl,productLabel.productLabelText,productLabel.spotlightLabel,lozengeModel.matchingLozenges
0,163153346,4,4.0,15,1,0,A spacious four bedroom lateral apartment to r...,"Warwick Gardens, London, W14.",GB,Flat,False,True,None,rent,False,False,True,False,False,True,Please note that the material information incl...,213 sq. m.,True,/properties/163153346#/?channel=RES_LET,/property-to-rent/contactBranch.html?propertyI...,None,RENT,2025-06-11T12:16:01Z,[],no_keyword,False,False,False,True,,None,None,None,Featured Property,False,False,Reduced on 03/04/2025,"by Knight Frank - Lettings, Kensington",,4 bedroom flat,51.495757,-0.202902,[{'url': '66k/65890/163153346/65890_KEQ0125390...,https://media.rightmove.co.uk:443/dir/crop/10:...,https://media.rightmove.co.uk:443/dir/crop/10:...,price_reduced,2025-04-03T16:16:36Z,2150,weekly,GBP,"[{'displayPrice': '£9,317 pcm', 'displayPriceQ...",65890,/company/clogo_rmchoice_16112_0017.png,020 3869 5439,"Knight Frank - Lettings, Kensington",Kensington,Knight Frank - Lettings,/estate-agents/agent/Knight-Frank---Lettings/K...,False,True,False,True,False,None,False,[],https://media.rightmove.co.uk:443/company/clog...,None,False,[]


### Filter out only the desired columns

In [26]:
def filter_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Filters the input DataFrame to retain only the columns relevant for property analysis.

    Args:
        df (pd.DataFrame): The DataFrame to filter.

    Returns:
        pd.DataFrame: A DataFrame containing only the selected columns of interest.
    """
    # Define the list of columns to keep in the filtered DataFrame
    base_cols = [
        'id',
        'bedrooms',
        'bathrooms',
        'numberOfImages',
        'displayAddress',
        'location.latitude',
        'location.longitude',
        'propertySubType',
        'listingUpdate.listingUpdateReason',
        'listingUpdate.listingUpdateDate',
        'price.amount',
        'price.frequency',
        'premiumListing',
        'featuredProperty',
        'transactionType',
        'students',
        'displaySize',
        'propertyUrl',
        'firstVisibleDate',
        'addedOrReduced',
        'propertyTypeFullDescription'
    ]
    # Assign the columns of interest (can be extended or modified if needed)
    columns_of_interest = base_cols
    # Filter the DataFrame to include only the columns of interest
    filtered_df = df[columns_of_interest]
    # Create a price per bedroom column
    filtered_df = filtered_df.copy()
    filtered_df.loc[:, "price_per_bed"] = filtered_df["price.amount"] / filtered_df["bedrooms"]
    # remove rows with a duplicated id
    filtered_df = filtered_df.drop_duplicates(subset="id")
    # Return the filtered DataFrame
    return filtered_df


filtered_df = filter_df(data_norm)
filtered_df.head()

,id,bedrooms,bathrooms,numberOfImages,displayAddress,location.latitude,location.longitude,propertySubType,listingUpdate.listingUpdateReason,listingUpdate.listingUpdateDate,price.amount,price.frequency,premiumListing,featuredProperty,transactionType,students,displaySize,propertyUrl,firstVisibleDate,addedOrReduced,propertyTypeFullDescription,price_per_bed
0,163153346,4,4.0,15,"Warwick Gardens, London, W14.",51.495757,-0.202902,Flat,price_reduced,2025-04-03T16:16:36Z,2150,weekly,False,True,rent,False,213 sq. m.,/properties/163153346#/?channel=RES_LET,2025-06-11T12:16:01Z,Reduced on 03/04/2025,4 bedroom flat,537.5
1,164031689,0,1.0,16,"Landmark Pinnacle, 10 Marsh Wall. Canary Wharf",51.502685,-0.025471,Apartment,new,2025-07-01T18:14:03Z,623,weekly,False,False,rent,False,42 sq. m.,/properties/164031689#/?channel=RES_LET,2025-07-01T18:08:35Z,Added today,Studio apartment,inf
2,163307378,3,1.0,15,"Forest Hill Road, London, SE22",51.452835,-0.060605,Semi-Detached,price_reduced,2025-07-01T18:13:54Z,3600,monthly,False,False,rent,False,,/properties/163307378#/?channel=RES_LET,2025-06-13T16:58:03Z,Reduced today,3 bedroom semi-detached house,1200.0
3,162735257,2,2.0,9,"Evering Road, London, E5",51.560450,-0.062852,Flat,price_reduced,2025-07-01T18:13:50Z,2635,monthly,False,False,rent,False,,/properties/162735257#/?channel=RES_LET,2025-06-02T15:12:55Z,Reduced today,2 bedroom flat,1317.5
4,161254532,3,1.0,8,"Springdale Road, London, N16",51.555540,-0.087071,Flat,price_reduced,2025-07-01T18:13:08Z,3150,monthly,False,False,rent,False,,/properties/161254532#/?channel=RES_LET,2025-04-29T10:23:18Z,Reduced today,3 bedroom flat,1050.0


In [27]:
filtered_df = rental_utils.functions.filter_df(df=filtered_df)


### Define a Function that Cleans Certain Column Names

In [28]:
def clean_column_names(df: pd.DataFrame) -> pd.DataFrame:
    """
    Renames selected columns of a DataFrame to make them more readable and SQL-friendly.
    Specifically, it replaces nested JSON column names (with dots) with simpler names.
    """
    # Create a mapping of the columns whose names we are changing
    rename_map = {
        "location.latitude": "latitude",
        "location.longitude": "longitude",
        "listingUpdate.listingUpdateReason": "listingUpdateReason",
        "listingUpdate.listingUpdateDate": "listingUpdateDate",
        "price.amount": "priceAmount",
        "price.frequency": "priceFrequency",
    }
    # then, actually rename the columns
    return df.rename(columns=rename_map)

In [29]:
clean_df = rental_utils.functions.clean_column_names(filtered_df)
#clean_df = clean_column_names(filtered_df)
clean_df.dtypes

id                               int64
bedrooms                         int64
bathrooms                      float64
numberOfImages                   int64
displayAddress                  object
latitude                       float64
longitude                      float64
propertySubType                 object
listingUpdateReason             object
listingUpdateDate               object
priceAmount                      int64
priceFrequency                  object
premiumListing                    bool
featuredProperty                  bool
transactionType                 object
students                          bool
displaySize                     object
propertyUrl                     object
firstVisibleDate                object
addedOrReduced                  object
propertyTypeFullDescription     object
price_per_bed                  float64
dtype: object

filtered_df.to_csv("../data/properties.csv")

### Create a Database and Save The Data to It

#### Create the Database if It Has Not Been Already

In [30]:
# Import as a separate sub-module to reduce the lenght of text
from rental_utils import sql_queries as sqlq

engine = sqlq.get_sql_engine("../data/properties.db")

with engine.connect() as conn:
    pass

#### Save the Data the Database as A New Table

In [31]:


## Execute the drop table query if needed (Drop the old table and start a fresh one)
with engine.connect() as connection:
    connection.execute(text(sqlq.DROP_PROPERTIES_TABLE_SQL_QUERY))

## Execute the CREATE TABLE query to create a blank table
with engine.connect() as connection:
    connection.execute(text(sqlq.CREATE_TABLE_SQL_QUERY))




In [32]:
## Save the dataframe into that table, extending it by default
sqlq.make_table(clean_df, "properties_data", engine)

### Check if it can be Extracted OK

In [33]:
with engine.connect() as connection:
    rightmove_data = pd.read_sql(text(sqlq.GET_PROPERTIES_DATA_SQL_QUERY), connection)
rightmove_data.head()

,id,price_per_bed,predicted_price_per_bed,travel_time,distance,bedrooms,bathrooms,numberOfImages,displayAddress,latitude,longitude,propertySubType,listingUpdateReason,listingUpdateDate,priceAmount,priceFrequency,premiumListing,featuredProperty,transactionType,students,displaySize,propertyUrl,firstVisibleDate,addedOrReduced,propertyTypeFullDescription
0,50425854,600.0,None,None,None,1,1.0,29,"St. George Wharf, London, SW8",51.486140,-0.125480,Apartment,new,2025-07-01T15:04:28Z,600,weekly,0,0,rent,1,52 sq. m.,/properties/50425854#/?channel=RES_LET,2017-09-07T14:00:35Z,Added today,1 bedroom apartment
1,80209919,246.0,None,None,None,3,1.0,22,"Dunton Road, London, SE1",51.492600,-0.074660,Terraced,new,2025-07-01T15:46:11Z,738,weekly,0,0,rent,0,70 sq. m.,/properties/80209919#/?channel=RES_LET,2019-03-18T14:08:31Z,Added today,3 bedroom terraced house
2,82567577,669.0,None,None,None,1,1.0,26,"Riverlight Quay, London, SW8",51.480250,-0.133930,Apartment,new,2025-07-01T14:57:09Z,669,weekly,0,0,rent,0,47 sq. m.,/properties/82567577#/?channel=RES_LET,2019-06-13T16:25:32Z,Added today,1 bedroom apartment
3,83161844,1275.0,None,None,None,2,2.0,7,"Greyhound Road, London, W6",51.484998,-0.218458,Apartment,new,2025-07-01T15:45:11Z,2550,monthly,0,0,rent,0,90 sq. m.,/properties/83161844#/?channel=RES_LET,2019-07-09T02:03:16Z,Added today,2 bedroom apartment
4,86137431,460.0,None,None,None,2,2.0,22,"The Modern, Embassy Gardens, London, SW11",51.481964,-0.130297,Apartment,new,2025-07-01T17:30:02Z,920,weekly,0,0,rent,0,,/properties/86137431#/?channel=RES_LET,2023-05-17T09:25:04Z,Added today,2 bedroom apartment
